<a href="https://colab.research.google.com/github/Ishikaaa/PDF-extraction/blob/main/gpt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# gpt-2 implementation problem -
# 1. https://github.com/openai/gpt-2/issues/

https://openai.com/index/better-language-models/

# Text Generation -

1.   https://github.com/laxmimerit/NLP-Tutorials-with-HuggingFace/blob/main/3%20Text%20Generation%20%7C%20NLP%20with%20HuggingFace%20Tutorial.ipynb
2.   https://www.youtube.com/watch?v=7O5ZKtJ99lc

In [4]:
!pip install PyPDF2
!pip install langchain
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install faiss-gpu
!pip install -U langchain-community

In [5]:
# import libraries
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

from langchain.llms import HuggingFacePipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import sys

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from langchain.chains import RetrievalQA

In [6]:
# Step-1: get text from single OR multiple PDFs
def get_pdf_text(pdf_docs):
    """
    args:
        pdf_docs: list of pdfs
    """
    text = ""

    # iterate through all pdfs
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        # iterate through all pages
        for page in pdf_reader.pages:
            text += page.extract_text()

    return text


# Step-2: get the text chunks
def get_text_chunks(text):
    """
    : return
        a list of chunks of text that we will feed to our model
    """
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = text_splitter.split_text(text)
    return chunks


# Step-3: Create Vector store
def get_vectorstore(text_chunks):
    # instructor embeddings
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

# def train_model():
#     model_id = "gpt2"
#     # model_id = "bertforquestionanswering"
#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     model = AutoModelForCausalLM.from_pretrained(model_id)

#     pipe = pipeline(
#     "question-answering", model=model, tokenizer=tokenizer, max_new_tokens=10
#     )

#     llm = HuggingFacePipeline(pipeline=pipe)

#     return llm

# from transformers import AutoTokenizer, AutoModelForQuestionAnswering
# from langchain.chains import RetrievalQA

def train_model():
    # Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
    model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

    model_name = "Intel/dynamic_tinybert"
    # Load the tokenizer associated with the specified model
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

    # Define a question-answering pipeline using the model and tokenizer
    question_answerer = pipeline(
        "question-answering",
        model=model_name,
        tokenizer=tokenizer,
        return_tensors='pt'
    )

    # Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
    # with additional model-specific arguments (temperature and max_length)
    llm = HuggingFacePipeline(
        pipeline=question_answerer,
        model_kwargs={"temperature": 0.7, "max_length": 512},
    )

    retriever = vector_store.as_retriever(search_kwargs={"k": 4})
    docs = retriever.get_relevant_documents("What is encoder and decoder stacks?")

    # Create a question-answering instance (qa) using the RetrievalQA class.
    # It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)
    return qa

def step5(qa):
    question = "Who is encoder and decoder stacks?"

    try:
        result = qa.invoke({"query": question})
        print(result["result"])
    except:
        def err_remove(er):
            lin = "------------"
            er = str(er)
            start_index = er.find(lin) + len(lin)
            end_index = er.rfind(lin)
            Answer = er[start_index:end_index].strip()
            return Answer
        _,error,_ = sys.exc_info()
        answer = err_remove(error)
        print(answer)


In [7]:
if __name__ == "__main__":
    # Step-1 Load PDF
    pdf_docs = ["NIPS-2017-attention-is-all-you-need-Paper.pdf"]
    raw_text = get_pdf_text(pdf_docs)

    # Step-2: get the text chunks
    text_chunks = get_text_chunks(raw_text)

    # Step-3
    vector_store = get_vectorstore(text_chunks)
    print("text_chunks: ", vector_store)

    # Step-4
    qa = train_model()

    # Step-5
    step5(qa)


load INSTRUCTOR_Transformer
max_seq_length  512
text_chunks:  <langchain_community.vectorstores.faiss.FAISS object at 0x7cf133de6a10>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_dep

Here, the encoder maps an input sequence of symbol representations (x1;:::;x n)to a sequence
of continuous representations z= (z1;:::;z n). Given z, the decoder then generates an output
sequence (y1;:::;y m)of symbols one element at a time. At each step the model is auto-regressive
[9], consuming the previously generated symbols as additional input when generating the next.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The ﬁrst is a multi-head self-attention mechanism, and the second is a simple, position-
2Figure 1: The Transformer - model architecture.
wise fully connected feed-forward network. We employ a residual connection [ 10] around each of
